In [3]:
#imports
try:
    import re
    import json
    import codecs
    from pprint import pprint
    from bs4 import BeautifulSoup
    from urllib.request import quote
    from urllib.request import urlopen
except ImportError:
    print("Error while loading libraries...")
except Exception:
    print('Uncaught exception')
else:
    print("Libraries loaded.")

Libraries loaded.


In [4]:
#constants
mainsite = "http://rozklad.nau.edu.ua/timetable/group/"
NNIIDS121 = "http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20121"
lessons = []

In [5]:
nauMain = urlopen(mainsite)
soupMain = BeautifulSoup(nauMain, 'html.parser')

In [6]:
groupTags = list(map(lambda x: x.text, soupMain.findChildren('select', attrs={'name': 'institute'})))

In [7]:
groupNames = (groupTags[0].split('\n'))[2:-1]

In [8]:
print(groupNames)

['ІІКАО', 'ДП  ННІН', 'ЗДН ННІН', 'ННІІДС', 'ННІАН', 'ННІАП', 'ННІЕБ', 'ННІЕМ', 'ННІКІТ', 'ННІМВ', 'ННАКІ', 'ННГМІ', 'ННЮІ', 'ПДН ННІН', 'ФТТ', 'ЦМО', 'ЦНТ', 'ЦНТ н-1']


In [9]:
groupNamesEncoded = list(map(lambda x: quote(x.encode('UTF-8')), groupNames))

In [10]:
print(groupNamesEncoded)

['%D0%86%D0%86%D0%9A%D0%90%D0%9E', '%D0%94%D0%9F%20%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%97%D0%94%D0%9D%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1', '%D0%9D%D0%9D%D0%86%D0%90%D0%9D', '%D0%9D%D0%9D%D0%86%D0%90%D0%9F', '%D0%9D%D0%9D%D0%86%D0%95%D0%91', '%D0%9D%D0%9D%D0%86%D0%95%D0%9C', '%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2', '%D0%9D%D0%9D%D0%86%D0%9C%D0%92', '%D0%9D%D0%9D%D0%90%D0%9A%D0%86', '%D0%9D%D0%9D%D0%93%D0%9C%D0%86', '%D0%9D%D0%9D%D0%AE%D0%86', '%D0%9F%D0%94%D0%9D%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%A4%D0%A2%D0%A2', '%D0%A6%D0%9C%D0%9E', '%D0%A6%D0%9D%D0%A2', '%D0%A6%D0%9D%D0%A2%20%D0%BD-1']


In [11]:
try:
    nau = urlopen(NNIIDS121)
except ConnectionError:
    print('ConnectionError')
except NameError:
    print('NameError')
except Exception:
    print('Uncaught exception')
else:
    print('HTTPResponse object loaded to variable')

HTTPResponse object loaded to variable


In [12]:
allScrapedGroups = []
def scrapeGroups():
    for s in groupNamesEncoded:
        print(mainsite+s)
        nau = urlopen(mainsite+s)
        allScrapedGroups.append(BeautifulSoup(nau, 'html.parser'))


In [13]:
scrapeGroups()

http://rozklad.nau.edu.ua/timetable/group/%D0%86%D0%86%D0%9A%D0%90%D0%9E
http://rozklad.nau.edu.ua/timetable/group/%D0%94%D0%9F%20%20%D0%9D%D0%9D%D0%86%D0%9D
http://rozklad.nau.edu.ua/timetable/group/%D0%97%D0%94%D0%9D%20%D0%9D%D0%9D%D0%86%D0%9D
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%9D
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%90%D0%9F
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%95%D0%91
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%95%D0%9C
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%9C%D0%92
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%90%D0%9A%D0%86
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%93%D0%9C%D0%86
http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%AE%D0%86
http://rozklad.nau

In [14]:
try:
    soup = BeautifulSoup(nau, 'html.parser')
except ImportError:
    print('ImportError')

In [51]:
week1 = soup.findAll('td', attrs={'colspan': '3'})[1:37]
week2 = soup.findAll('td', attrs={'colspan': '3'})[38:-1]
# x =     soup.findAll('td', attrs={'colspan': '3'})[1:]
x = week1+week2

In [52]:
print(len(x))
print(x[70])

71
<td colspan="3"></td>


In [17]:
# 1. -> 08:00 - 09:35	15
# 2. -> 09:50 - 11:25	15
# 3. -> 11:40 - 13:15	15
# 4. -> 13:30 - 15:05	15
# 5. -> 15:20 - 16:55	15
# 6. -> 17:10 - 18:45	15
# 7. -> 19:00 - 20:35	15
# 8. -> 20:50 - 22:25


def paraToTime(num):
    schedule = [
        "08:00 - 09:35",
        "09:50 - 11:25",
        "11:40 - 13:15",
        "13:30 - 15:05",
        "15:20 - 16:55",
        "17:10 - 18:45",
        "19:00 - 20:35",
        "20:50 - 22:25"

    ]
    return schedule[num]

In [18]:
paraToTime(0)

'08:00 - 09:35'

In [75]:

def makeObj(i,tag):
    if not tag.contents: 
#         print("NOT-----")
        return {}
    a=0
    if i < 35:
       a = 1
    else: 
        a = 0
    index = tag.small.contents[0].find(' ')
    return {
        'name': tag.b.contents[0], 
        'short_name': tag.b.contents[0], 
        'lesson_time':paraToTime(i%6),
        'day': int(i/6)+1,
        'format': lessonFormat(tag),
        'teachers': [{
                  "degree":-1,
                  "last_name": tag.small.contents[0][0:index],
                  "first_name": tag.small.contents[0][index+1:index+2],
                  "middle_name": tag.small.contents[0][index+3:index+4]
               } ], 
        'group': tag.contents[4][35:48].rstrip(),
        'weeks':a,
        'housing':re.findall('\d+',tag.contents[0])[0],
        'room_num':re.findall('\d+',tag.contents[0])[1],
        'room_floor':re.findall('\d+',tag.contents[0])[1][0]}


In [76]:
def lessonFormat(tag):
    if 'class' in tag.attrs:
        return 'lecture'
    else:
        return 'practice'

In [77]:
if 'class' in x[8]:
    print("fdsa")
lessonFormat(x[8])

'lecture'

In [78]:
y = {z: makeObj(z,x[z]) for z in range(0,35)}

In [79]:
pprint(y)

{0: {'day': 1,
     'format': 'practice',
     'group': 'ННІІДС 121',
     'housing': '11',
     'lesson_time': '08:00 - 09:35',
     'name': 'Вища математика',
     'room_floor': '2',
     'room_num': '214',
     'short_name': 'Вища математика',
     'teachers': [{'degree': -1,
                   'first_name': 'С',
                   'last_name': 'Глоба',
                   'middle_name': 'О'}],
     'weeks': 1},
 1: {'day': 1,
     'format': 'practice',
     'group': 'ННІІДС 121',
     'housing': '11',
     'lesson_time': '09:50 - 11:25',
     'name': 'Історія та культура України',
     'room_floor': '2',
     'room_num': '214',
     'short_name': 'Історія та культура України',
     'teachers': [{'degree': -1,
                   'first_name': 'Н',
                   'last_name': 'Сенчило',
                   'middle_name': 'О'}],
     'weeks': 1},
 2: {},
 3: {},
 4: {},
 5: {},
 6: {},
 7: {},
 8: {'day': 2,
     'format': 'lecture',
     'group': 'ННІІДС 1к2п',
     'housing': '11'

In [80]:
# r = json.dumps(y, ensure_ascii=False)
k = {k:v for (k,v) in y.items() if len(v)>0}
with codecs.open("NAU.json", "w", encoding='utf-8') as out:
    json.dump(k, out, ensure_ascii=False, indent=4)

# result = json.loads(r)

In [114]:
# pprint(k)

In [ ]:
def getWeeks():
    week1 = soup.findAll('td', attrs={'colspan': '3'})[1:37]
    week2 = soup.findAll('td', attrs={'colspan': '3'})[38:-1]